In [ ]:
import os
os.getcwd()

: 

In [2]:
pip install lightgbm


   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   --- ------------------------------------ 0.1/1.4 MB 4.3 MB/s eta 0:00:01
   ------- -------------------------------- 0.3/1.4 MB 3.2 MB/s eta 0:00:01
   ---------- ----------------------------- 0.4/1.4 MB 3.0 MB/s eta 0:00:01
   ------------- -------------------------- 0.5/1.4 MB 2.8 MB/s eta 0:00:01
   ----------------- ---------------------- 0.6/1.4 MB 2.9 MB/s eta 0:00:01
   -------------------- ------------------- 0.7/1.4 MB 2.8 MB/s eta 0:00:01
   ------------------------ --------------- 0.9/1.4 MB 2.8 MB/s eta 0:00:01
   --------------------------- ------------ 1.0/1.4 MB 2.7 MB/s eta 0:00:01
   ------------------------------ --------- 1.1/1.4 MB 2.7 MB/s eta 0:00:01
   --------------------------------- ------ 1.2/1.4 MB 2.7 MB/s eta 0:00:01
   ------------------------------------- -- 1.3/1.4 MB 2.7 MB/s eta 0:00:01
   ---------------------------------------  1.4/1.4 MB 2.7 MB/s eta 0:00:01
   ----------------


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb # type: ignore
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load the data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
stores = pd.read_csv('stores.csv')
oil = pd.read_csv('oil.csv')
holidays_events = pd.read_csv('holidays_events.csv')

# Parse dates
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])
oil['date'] = pd.to_datetime(oil['date'])
holidays_events['date'] = pd.to_datetime(holidays_events['date'])

# Merge stores data
train = pd.merge(train, stores, on='store_nbr', how='left')
test = pd.merge(test, stores, on='store_nbr', how='left')

# Merge oil data
train = pd.merge(train, oil, on='date', how='left')
test = pd.merge(test, oil, on='date', how='left')

# Fill missing oil prices with forward fill
train['dcoilwtico'].fillna(method='ffill', inplace=True)
test['dcoilwtico'].fillna(method='ffill', inplace=True)

# Encode categorical features
le = LabelEncoder()
train['family'] = le.fit_transform(train['family'])
test['family'] = le.transform(test['family'])
train['city'] = le.fit_transform(train['city'])
test['city'] = le.transform(test['city'])
train['state'] = le.fit_transform(train['state'])
test['state'] = le.transform(test['state'])
train['type'] = le.fit_transform(train['type'])
test['type'] = le.transform(test['type'])

# Create date features
def create_date_features(df):
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['dayofweek'] = df['date'].dt.dayofweek
    df['is_weekend'] = df['dayofweek'] >= 5
    return df

train = create_date_features(train)
test = create_date_features(test)

# Incorporate holidays and events information
holidays_events = holidays_events[holidays_events['transferred'] == 'False']
holidays_events = holidays_events[['date', 'type', 'locale', 'locale_name']]
holidays_events['is_holiday'] = 1

train = pd.merge(train, holidays_events, on=['date'], how='left')
test = pd.merge(test, holidays_events, on=['date'], how='left')

train['is_holiday'].fillna(0, inplace=True)
test['is_holiday'].fillna(0, inplace=True)

# Prepare the data for training
X = train.drop(['date', 'sales'], axis=1)
y = train['sales']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the lightGBM dataset
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

# Specify the model parameters
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'mae'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

# Train the model
print('Starting training...')
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=1000,
                valid_sets=lgb_eval,
                early_stopping_rounds=10)

print('Training completed!')

# Predict on the validation set
y_pred = gbm.predict(X_val, num_iteration=gbm.best_iteration)

# Calculate evaluation metrics
mae = mean_absolute_error(y_val, y_pred)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))

print(f'Mean Absolute Error: {mae}')
print(f'Root Mean Squared Error: {rmse}')

# Prepare the test set for prediction
X_test = test.drop(['date'], axis=1)

# Predict on the test set
test['sales'] = gbm.predict(X_test, num_iteration=gbm.best_iteration)

# Prepare the submission file
submission = test[['id', 'sales']]
submission.to_csv('submission.csv', index=False)

print('Submission file created!')


C:\Users\soni\AppData\Local\Temp\ipykernel_21664\1741009627.py:30: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train['dcoilwtico'].fillna(method='ffill', inplace=True)
C:\Users\soni\AppData\Local\Temp\ipykernel_21664\1741009627.py:31: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test['dcoilwtico'].fillna(method='ffill', inplace=True)


Starting training...


TypeError: train() got an unexpected keyword argument 'early_stopping_rounds'

In [10]:
import lightgbm as lgb
print(lgb.__version__)


4.4.0


In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load the data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
stores = pd.read_csv('stores.csv')
oil = pd.read_csv('oil.csv')
holidays_events = pd.read_csv('holidays_events.csv')

# Parse dates
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])
oil['date'] = pd.to_datetime(oil['date'])
holidays_events['date'] = pd.to_datetime(holidays_events['date'])

# Merge stores data
train = pd.merge(train, stores, on='store_nbr', how='left')
test = pd.merge(test, stores, on='store_nbr', how='left')

# Merge oil data
train = pd.merge(train, oil, on='date', how='left')
test = pd.merge(test, oil, on='date', how='left')

# Fill missing oil prices with forward fill
train['dcoilwtico'].fillna(method='ffill', inplace=True)
test['dcoilwtico'].fillna(method='ffill', inplace=True)

# Encode categorical features
le = LabelEncoder()
train['family'] = le.fit_transform(train['family'])
test['family'] = le.transform(test['family'])
train['city'] = le.fit_transform(train['city'])
test['city'] = le.transform(test['city'])
train['state'] = le.fit_transform(train['state'])
test['state'] = le.transform(test['state'])
train['type'] = le.fit_transform(train['type'])
test['type'] = le.transform(test['type'])

# Create date features
def create_date_features(df):
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['dayofweek'] = df['date'].dt.dayofweek
    df['is_weekend'] = df['dayofweek'] >= 5
    return df

train = create_date_features(train)
test = create_date_features(test)

# Incorporate holidays and events information
holidays_events = holidays_events[holidays_events['transferred'] == 'False']
holidays_events = holidays_events[['date', 'type', 'locale', 'locale_name']]
holidays_events['is_holiday'] = 1

train = pd.merge(train, holidays_events, on=['date'], how='left')
test = pd.merge(test, holidays_events, on=['date'], how='left')

train['is_holiday'].fillna(0, inplace=True)
test['is_holiday'].fillna(0, inplace=True)

# Prepare the data for training
X = train.drop(['date', 'sales'], axis=1)
y = train['sales']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the lightGBM dataset
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

# Specify the model parameters
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'mae'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

# Train the model
print('Starting training...')
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=1000,
                valid_sets=[lgb_eval],
                early_stopping_rounds=10)

print('Training completed!')

# Predict on the validation set
y_pred = gbm.predict(X_val, num_iteration=gbm.best_iteration)

# Calculate evaluation metrics
mae = mean_absolute_error(y_val, y_pred)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))

print(f'Mean Absolute Error: {mae}')
print(f'Root Mean Squared Error: {rmse}')

# Prepare the test set for prediction
X_test = test.drop(['date'], axis=1)

# Predict on the test set
test['sales'] = gbm.predict(X_test, num_iteration=gbm.best_iteration)

# Prepare the submission file
submission = test[['id', 'sales']]
submission.to_csv('submission.csv', index=False)

print('Submission file created!')


C:\Users\soni\AppData\Local\Temp\ipykernel_21664\4063752079.py:30: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train['dcoilwtico'].fillna(method='ffill', inplace=True)
C:\Users\soni\AppData\Local\Temp\ipykernel_21664\4063752079.py:31: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test['dcoilwtico'].fillna(method='ffill', inplace=True)


Starting training...


TypeError: train() got an unexpected keyword argument 'early_stopping_rounds'

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load the data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
stores = pd.read_csv('stores.csv')
oil = pd.read_csv('oil.csv')
holidays_events = pd.read_csv('holidays_events.csv')

# Parse dates
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])
oil['date'] = pd.to_datetime(oil['date'])
holidays_events['date'] = pd.to_datetime(holidays_events['date'])

# Merge stores data
train = pd.merge(train, stores, on='store_nbr', how='left')
test = pd.merge(test, stores, on='store_nbr', how='left')

# Merge oil data
train = pd.merge(train, oil, on='date', how='left')
test = pd.merge(test, oil, on='date', how='left')

# Fill missing oil prices with forward fill
train['dcoilwtico'].fillna(method='ffill', inplace=True)
test['dcoilwtico'].fillna(method='ffill', inplace=True)

# Encode categorical features
le = LabelEncoder()
categorical_cols = ['family', 'city', 'state', 'type', 'locale', 'locale_name']
for col in categorical_cols:
    if col in train.columns:
        train[col] = le.fit_transform(train[col])
        test[col] = le.transform(test[col])

# Create date features
def create_date_features(df):
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['dayofweek'] = df['date'].dt.dayofweek
    df['is_weekend'] = df['dayofweek'] >= 5
    return df

train = create_date_features(train)
test = create_date_features(test)

# Incorporate holidays and events information
holidays_events = holidays_events[holidays_events['transferred'] == 'False']
holidays_events = holidays_events[['date', 'type', 'locale', 'locale_name']]
holidays_events['is_holiday'] = 1

train = pd.merge(train, holidays_events, on=['date'], how='left')
test = pd.merge(test, holidays_events, on=['date'], how='left')

train['is_holiday'].fillna(0, inplace=True)
test['is_holiday'].fillna(0, inplace=True)

# Ensure all columns are numerical
train = train.apply(pd.to_numeric, errors='coerce')
test = test.apply(pd.to_numeric, errors='coerce')

# Prepare the data for training
X = train.drop(['date', 'sales'], axis=1)
y = train['sales']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the XGBoost model
import joblib # type: ignore
model = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=7,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
model.fit(X_train, y_train, early_stopping_rounds=10, eval_set=[(X_val, y_val)], verbose=True)
joblib.dump(model, 'xgb_model.pkl')


# Train the model
#print('Starting training...')
#model.fit(X_train, y_train, early_stopping_rounds=10, eval_set=[(X_val, y_val)], verbose=True)

print('Training completed!')

# Predict on the validation set
#y_pred = model.predict(X_val)

# Calculate evaluation metrics
mae = mean_absolute_error(y_val, y_pred)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))

print(f'Mean Absolute Error: {mae}')
print(f'Root Mean Squared Error: {rmse}')

# Prepare the test set for prediction
X_test = test.drop(['date'], axis=1)

# Predict on the test set
#test['sales'] = model.predict(X_test)
# Visualize predicted vs actual sales
plt.figure(figsize=(10, 6))
plt.scatter(y_val, y_pred, color='blue', alpha=0.5)
plt.plot([min(y_val), max(y_val)], [min(y_val), max(y_val)], 'k--', lw=2)
plt.xlabel('Actual Sales')
plt.ylabel('Predicted Sales')
plt.title('Actual vs Predicted Sales')
plt.grid(True)
plt.show()


# Prepare the submission file
#submission = test[['id', 'sales']]
#submission.to_csv('submission.csv', index=False)

print('Submission file created!')


C:\Users\soni\AppData\Local\Temp\ipykernel_3404\4159561846.py:31: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train['dcoilwtico'].fillna(method='ffill', inplace=True)
C:\Users\soni\AppData\Local\Temp\ipykernel_3404\4159561846.py:32: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test['dcoilwtico'].fillna(method='ffill', inplace=True)
c:\Users\soni\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-rmse:1092.02340
[1]	validation_0-rmse:1054.23298
[2]	validation_0-rmse:1034.49973
[3]	validation_0-rmse:999.13249
[4]	validation_0-rmse:981.97490
[5]	validation_0-rmse:965.88716
[6]	validation_0-rmse:934.37176
[7]	validation_0-rmse:904.98566
[8]	validation_0-rmse:873.75219
[9]	validation_0-rmse:847.87542
[10]	validation_0-rmse:820.10998
[11]	validation_0-rmse:796.43963
[12]	validation_0-rmse:771.52420
[13]	validation_0-rmse:751.22358
[14]	validation_0-rmse:732.11501
[15]	validation_0-rmse:723.22045
[16]	validation_0-rmse:715.04138
[17]	validation_0-rmse:713.12877
[18]	validation_0-rmse:695.70444
[19]	validation_0-rmse:679.44337
[20]	validation_0-rmse:673.07672
[21]	validation_0-rmse:658.39681
[22]	validation_0-rmse:652.64244
[23]	validation_0-rmse:647.20525
[24]	validation_0-rmse:634.58355
[25]	validation_0-rmse:622.27882
[26]	validation_0-rmse:610.90322
[27]	validation_0-rmse:600.43872
[28]	validation_0-rmse:591.12634
[29]	validation_0-rmse:582.00998
[30]	validation_0

NameError: name 'y_pred' is not defined

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load the data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
stores = pd.read_csv('stores.csv')
oil = pd.read_csv('oil.csv')
holidays_events = pd.read_csv('holidays_events.csv')

# Ensure 'date' columns are datetime64[ns]
for df in [train, test, oil, holidays_events]:
    df['date'] = pd.to_datetime(df['date'])

# Merge stores data
train = pd.merge(train, stores, on='store_nbr', how='left')
test = pd.merge(test, stores, on='store_nbr', how='left')

# Merge oil data
train = pd.merge(train, oil, on='date', how='left')
test = pd.merge(test, oil, on='date', how='left')

# Fill missing oil prices with forward fill
train['dcoilwtico'].fillna(method='ffill', inplace=True)
test['dcoilwtico'].fillna(method='ffill', inplace=True)

# Encode categorical features
le = LabelEncoder()
categorical_cols = ['family', 'city', 'state', 'type', 'locale', 'locale_name']
for col in categorical_cols:
    if col in train.columns:
        train[col] = le.fit_transform(train[col])
        test[col] = le.transform(test[col])

# Create date features
def create_date_features(df):
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['dayofweek'] = df['date'].dt.dayofweek
    df['is_weekend'] = df['dayofweek'] >= 5
    return df

train = create_date_features(train)
test = create_date_features(test)

# Incorporate holidays and events information
holidays_events = holidays_events[holidays_events['transferred'] == 'False']
holidays_events = holidays_events[['date', 'type', 'locale', 'locale_name']]
holidays_events['is_holiday'] = 1

train = pd.merge(train, holidays_events, on=['date'], how='left')
test = pd.merge(test, holidays_events, on=['date'], how='left')

train['is_holiday'].fillna(0, inplace=True)
test['is_holiday'].fillna(0, inplace=True)

# Ensure all columns are numerical
train = train.apply(pd.to_numeric, errors='coerce')
test = test.apply(pd.to_numeric, errors='coerce')

# Prepare the data for training
X = train.drop(['date', 'sales'], axis=1)
y = train['sales']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the XGBoost model
model = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=7,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
model.fit(X_train, y_train, early_stopping_rounds=10, eval_set=[(X_val, y_val)], verbose=True)
joblib.dump(model, 'xgb_model1.pkl')
# Train the model
print('Starting training...')
model.fit(X_train, y_train, early_stopping_rounds=10, eval_set=[(X_val, y_val)], verbose=True)
print('Training completed!')

# Predict on the validation set
y_pred = model.predict(X_val)

# Calculate evaluation metrics
mae = mean_absolute_error(y_val, y_pred)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))

print(f'Mean Absolute Error: {mae}')
print(f'Root Mean Squared Error: {rmse}')

# Prepare the test set for prediction
X_test = test.drop(['date'], axis=1)

# Predict on the test set
test['sales'] = model.predict(X_test)

# Visualize predicted vs actual sales
plt.figure(figsize=(10, 6))
plt.scatter(y_val, y_pred, color='blue', alpha=0.5)
plt.plot([min(y_val), max(y_val)], [min(y_val), max(y_val)], 'k--', lw=2)
plt.xlabel('Actual Sales')
plt.ylabel('Predicted Sales')
plt.title('Actual vs Predicted Sales')
plt.grid(True)
plt.show()

# Prepare the submission file
#submission = test[['id', 'sales']]
#submission.to_csv('submission.csv', index=False)

print('Submission file created!')


C:\Users\soni\AppData\Local\Temp\ipykernel_3404\69574060.py:29: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train['dcoilwtico'].fillna(method='ffill', inplace=True)
C:\Users\soni\AppData\Local\Temp\ipykernel_3404\69574060.py:30: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test['dcoilwtico'].fillna(method='ffill', inplace=True)
